In [2]:
import cv2 as cv 
import numpy as np
import pandas as pd

import os
import glob

In [3]:
X = []
Y = []
DIR = r"E:\Dataset"

classifier = cv.CascadeClassifier(cv.data.haarcascades + "haarcascade_frontalface_default.xml")

In [4]:

# def extract_face_roi_means(frame):
#     gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
#     green = frame[:, :, 1]  # Extract green channel

#     faces = classifier.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=3)
#     if len(faces) == 0:
#         return None

#     x, y, w, h = faces[0]  # Use the first face

#     # Define ROIs based on face box proportions
#     roi_means = 0

#     # Forehead region (top center)
#     fh_x1 = x + int(0.3 * w)
#     fh_x2 = x + int(0.7 * w)
#     fh_y1 = y + int(0.05 * h)
#     fh_y2 = y + int(0.25 * h)
#     forehead = green[fh_y1:fh_y2, fh_x1:fh_x2]
#     roi_means+= np.mean(forehead)

#     # Left cheek
#     lc_x1 = x + int(0.05 * w)
#     lc_x2 = x + int(0.3 * w)
#     lc_y1 = y + int(0.5 * h)
#     lc_y2 = y + int(0.75 * h)
#     left_cheek = green[lc_y1:lc_y2, lc_x1:lc_x2]
#     roi_means+= np.mean(left_cheek)

#     # Right cheek
#     rc_x1 = x + int(0.7 * w)
#     rc_x2 = x + int(0.95 * w)
#     rc_y1 = lc_y1
#     rc_y2 = lc_y2
#     right_cheek = green[rc_y1:rc_y2, rc_x1:rc_x2]
#     roi_means+= np.mean(right_cheek)

#     # Chin
#     ch_x1 = x + int(0.4 * w)
#     ch_x2 = x + int(0.6 * w)
#     ch_y1 = y + int(0.8 * h)
#     ch_y2 = y + int(0.95 * h)
#     chin = green[ch_y1:ch_y2, ch_x1:ch_x2]
#     roi_means+= np.mean(chin)

#     return roi_means/4


In [ ]:
for x in os.listdir(DIR):
    folder_path = os.path.join(DIR, x)
    if not os.path.isdir(folder_path):
        continue

    video_list = glob.glob(os.path.join(folder_path, '*.avi'))
    csv_list = glob.glob(os.path.join(folder_path, '*.csv'))

    if not video_list or not csv_list:
        continue

    video = video_list[0]
    csv = csv_list[0]

    cap = cv.VideoCapture(video)
    df = pd.read_csv(csv)

    fps = cap.get(cv.CAP_PROP_FPS)
    
    X_feature = []
    Y_feature = []

    total_rows = len(df)
    usable_rows = total_rows - (total_rows % 24)

    for i in range(usable_rows):
        timestamp_ms = df["Timestep (ms)"][i]
        frame_number = int((timestamp_ms / 1000.0) * fps)
        cap.set(cv.CAP_PROP_POS_FRAMES, frame_number)

        ret, frame = cap.read()
        if not ret:
            break

        gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        green = frame[:, :, 1]

        faces = classifier.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=3)
        if len(faces) == 0:
            continue

        x, y, w, h = faces[0]
        face_green = green[y:y+h, x:x+w]
        mean_g = np.mean(face_green)

        X_feature.append([mean_g])
        Y_feature.append([df["Heart rate (HR)"][i]])

    b = int(len(X_feature) / 24)
    X_feature = np.array(X_feature[:b*24]).reshape(b, 24, 1)
    Y_feature = np.array(Y_feature[:b*24]).reshape(b, 24)

    X.append(X_feature)
    Y.append(Y_feature)


In [6]:
# for x in os.listdir(DIR):
#     folder_path = os.path.join(DIR, x)
#     if not os.path.isdir(folder_path):
#         continue

#     video_list = glob.glob(os.path.join(folder_path, '*.avi'))
#     csv_list = glob.glob(os.path.join(folder_path, '*.csv'))

#     if not video_list or not csv_list:
#         continue

#     video = video_list[0]
#     csv = csv_list[0]

#     cap = cv.VideoCapture(video)
#     df = pd.read_csv(csv)

#     fps = cap.get(cv.CAP_PROP_FPS)
    
#     X_feature = []
#     Y_feature = []

#     total_rows = len(df)
#     usable_rows = total_rows - (total_rows % 24)

#     for i in range(usable_rows):
#         timestamp_ms = df["Timestep (ms)"][i]
#         frame_number = int((timestamp_ms / 1000.0) * fps)
#         cap.set(cv.CAP_PROP_POS_FRAMES, frame_number)

#         ret, frame = cap.read()
#         if not ret:
#             break
        
#         mean_g = extract_face_roi_means(frame)

#         X_feature.append([mean_g])
#         Y_feature.append([df["Heart rate (HR)"][i]])

#     b = int(len(X_feature) / 24)
#     X_feature = np.array(X_feature[:b*24]).reshape(b, 24, 1)
#     Y_feature = np.array(Y_feature[:b*24]).reshape(b, 24)

#     X.append(X_feature)
#     Y.append(Y_feature)


In [ ]:
X = np.vstack(X)  # or np.concatenate(X, axis=0) depending on shape
Y = np.vstack(Y)

np.save("input1.npy", X)
np.save("output1.npy", Y)


In [4]:
# len(Y)

In [5]:
Y

NameError: name 'Y' is not defined

In [ ]:
out=np.mean(Y,axis=1).astype("int64")
out=out.reshape(-1,1)

In [ ]:
out

array([[ 75],
       [ 75],
       [ 75],
       ...,
       [100],
       [ 99],
       [ 99]], dtype=int64)

In [ ]:
np.save("out1.npy", out)

In [ ]:
len(out)

1560

In [ ]:
len(X)

1560

In [ ]:
X

In [ ]:
df=pd.read_csv(r"E:\Dataset-2\subject8\ground_truth.txt",delimiter="/t")

C:\Users\HP\AppData\Local\Temp\ipykernel_18480\3053672303.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df=pd.read_csv(r"E:\Dataset-2\subject8\ground_truth.txt",delimiter="/t")


In [ ]:
df

In [ ]:
with open('E:\Dataset-2\subject8\ground_truth.txt', 'r') as f:
    lines = f.readlines()

ppg_values = list(map(float, lines[0].strip().split()))
hr_values = list(map(float, lines[1].strip().split()))
time_values = list(map(float, lines[2].strip().split()))

In [ ]:
ppg_values

[0.6814614,
 0.14493817,
 -0.11010429,
 -0.26087117,
 -0.28952585,
 -0.30008006,
 -0.46894741,
 -0.73280264,
 -1.0494289,
 -1.3343926,
 -1.4504889,
 -1.0599831,
 -0.26841743,
 0.73423244,
 1.1352924,
 1.086057,
 0.60758193,
 0.092167125,
 -0.18926086,
 -0.32118848,
 -0.35638677,
 -0.41163805,
 -0.57623094,
 -0.8066821,
 -1.0524369,
 -0.95444103,
 -0.03976049,
 0.64626311,
 1.4818398,
 1.7896534,
 1.5961419,
 1.367485,
 0.93650386,
 0.43079893,
 0.039396079,
 -0.13648982,
 -0.19807363,
 -0.31237571,
 -0.57623094,
 -0.92805551,
 -1.0424104,
 -0.30361572,
 0.88373281,
 1.5697564,
 1.3938705,
 0.90135834,
 0.34726236,
 0.10097989,
 -0.1804481,
 -0.38277229,
 -0.54984542,
 -0.67301304,
 -0.88409722,
 -1.0599831,
 -1.2007235,
 -1.2710673,
 -0.86399146,
 -0.16287534,
 0.80811191,
 1.3147139,
 1.6137675,
 1.5257981,
 1.1915991,
 0.94531663,
 0.54953378,
 0.18013646,
 -0.1206585,
 -0.30008006,
 -0.38715229,
 -0.51116424,
 -0.72578409,
 -0.98082655,
 -1.2446818,
 -1.455766,
 -1.408272,
 -0.66947

In [ ]:
print(len(time_values))
print(len(ppg_values))
print(len(hr_values))



2022
2022
2022


In [ ]:
hr_values

[97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 97.0,
 98.0,
 98.0,
 98.0,
 98.0,
 98.0,
 98.0,
 98.0,
 98.0,
 98.0,
 98.0,
 98.0,
 98.0,
 98.0,
 98.0,
 98.0,
 98.0,
 98.0,
 98.667,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 10

In [ ]:
time_values

[0.0,
 0.042,
 0.066,
 0.1,
 0.132,
 0.165,
 0.201,
 0.234,
 0.266,
 0.302,
 0.334,
 0.366,
 0.404,
 0.434,
 0.466,
 0.501,
 0.532,
 0.573,
 0.6,
 0.642,
 0.665,
 0.707,
 0.747,
 0.789,
 0.83,
 0.872,
 0.913,
 0.925,
 0.971,
 0.982,
 1.024,
 1.035,
 1.077,
 1.1,
 1.142,
 1.165,
 1.206,
 1.248,
 1.29,
 1.333,
 1.375,
 1.416,
 1.458,
 1.5,
 1.542,
 1.583,
 1.628,
 1.639,
 1.681,
 1.723,
 1.765,
 1.777,
 1.818,
 1.83,
 1.872,
 1.883,
 1.93,
 1.942,
 1.984,
 1.996,
 2.038,
 2.05,
 2.092,
 2.105,
 2.148,
 2.167,
 2.201,
 2.233,
 2.264,
 2.299,
 2.334,
 2.365,
 2.401,
 2.432,
 2.465,
 2.5,
 2.532,
 2.564,
 2.601,
 2.633,
 2.675,
 2.7,
 2.742,
 2.764,
 2.806,
 2.832,
 2.864,
 2.9,
 2.932,
 2.966,
 3.0,
 3.032,
 3.064,
 3.101,
 3.134,
 3.168,
 3.203,
 3.232,
 3.264,
 3.3,
 3.332,
 3.364,
 3.4,
 3.432,
 3.464,
 3.5,
 3.532,
 3.565,
 3.602,
 3.633,
 3.664,
 3.706,
 3.733,
 3.764,
 3.8,
 3.832,
 3.866,
 3.9,
 3.934,
 3.971,
 4.003,
 4.036,
 4.064,
 4.1,
 4.132,
 4.163,
 4.2,
 4.232,
 4.264,
 4.30